In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import linear_kernel
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [13]:
import pickle

In [2]:
df2=pd.read_csv("df_CA_Summarization_filtered.csv")

In [ ]:
df2.head()

,Unnamed: 0,Unnamed: 0.1,education,gPlusUserId,address,closed,gps,hours,name,phone,...,jobs,loc,cleaned_review,cleaned_summary,cleaned_education,cleaned_address,cleaned_hours,cleaned_categories,cleaned_jobs,sentiment_VADER
0,0,0,"[['[]'; '[]'; '[]'; '[]'; '[]'; '6']; ['[""The ...",1.002152e+20,['546 University Ave'; 'Palo Alto; CA 94301'],False,"[37.448889, -122.158481]","[['Monday'; '[[""11:30 am--2:30 pm""];[""5:00--9:...",Tamarine,(650) 325-8500,...,[['Google'; 'Engineer'; '[[];[1;1;2011];1]'; '...,California,bit expensive lunch specials reasonable excell...,bit expensive but lunch specials are reasonabl...,university minnesota computer science,546 university ave palo alto; ca 94301,monday 11:30 am; 2:30 pm ;5:00; 9:00 pm tuesda...,restaurant asian restaurant southeast asian re...,google engineer ;1;1;2011 ;1,0.7227
1,2,3,"[['[]'; '[]'; '[]'; '[]'; '[]'; '6']; ['[""Univ...",1.019804e+20,['655 Divisadero St'; 'San Francisco; CA 94117'],False,"[37.775721, -122.438054]","[['Monday'; '[[""Closed""]]'; '1']; ['Tuesday'; ...",BAR CRUDO,(415) 409-0679,...,[['Retailigence'; 'Director of Business Develo...,California,would normally consider foofoo based portion s...,the crudo is made up of pure deliciousness nei...,university missouri rolla computer engineering,655 divisadero st san francisco; ca 94117,monday closed tuesday 5:00; 10:00 pm wednesday...,restaurant bar seafood restaurant,retailigence director of business development ...,0.8519
2,3,7,"[['[]'; '[]'; '[]'; '[]'; '[]'; '6']; ['[""Univ...",1.029165e+20,['355 11th St'; 'San Francisco; CA 94103'],False,"[37.771291, -122.413113]","[['Monday'; '[[""6:00--10:00 pm""]]'; '1']; ['Tu...",Bar Agricole,(415) 355-9400,...,[['Farmer Brownstein Jaeger LLP'; 'Partner'; '...,California,drinks outshine food great place amiable evening,drinks outshine the food here but great place ...,university california berkeley http www freeba...,355 11th st san francisco; ca 94103,monday 6:00; 10:00 pm tuesday 6:00; 10:00 pm w...,bar restaurant,farmer brownstein jaeger llp partner,0.7684
3,5,16,"[['[]'; '[]'; '[]'; '[]'; '[]'; '6']; ['[""Texa...",1.060633e+20,['6530 Sunset Blvd'; 'Los Angeles; CA 90028'],False,"[34.097787, -118.332712]","[['Monday'; '[[""11:30 am--10:30 pm""]]']; ['Tue...",Cat and Fiddle Pub & Restaurant,(323) 468-3800,...,"[[""McDonald's""; 'Burger Bitch'; '[]'; 'http://...",California,brings back time spent uk patio great atmosphe...,the patio is great atmosphere don think would ...,texas university http www freebase com view ccs,6530 sunset blvd los angeles; ca 90028,monday 11:30 am; 10:30 pm tuesday 11:30 am; 11...,restaurant bar historical landmark,mcdonald; burger bitch http://www.freebase.com...,0.6249
4,6,18,[['[]'; '[]'; '[]'; '[]'; '[]'; '6']; []],1.074869e+20,['2298 Market St'; 'San Francisco; CA 94114'],False,"[37.76459, -122.432918]","[['Monday'; '[[""7:00 am--1:00 am""]]'; '1']; ['...",Cafe Flore,(415) 621-8579,...,[['YouTube'; 'Web Developer'; '[[1;1;2011];[1;...,California,standard goto meal anytime day great outdoor s...,great outdoor seating supposedly they have fre...,NaN,2298 market st san francisco; ca 94114,monday 7:00 am; 1:00 am tuesday 7:00 am; 1:00 ...,cafe restaurant coffee shop,youtube web developer 1;1;2011 ;1;1;2011 ;1 ve...,0.5719


In [ ]:
mapping = pd.Series(df2.index,index = df2['name'])
mapping

name
Tamarine                               0
Blue Fig                               1
Umami Burger                           2
BAR CRUDO                              3
Southern Pacific Brewing               4
                                    ... 
Naja's Place                        2084
Crow's Nest Restaurant              2090
The Kitchen Restaurant              2209
Olive Cafe                          2647
Espana's Southwest Bar and Grill    2703
Length: 347, dtype: int64

In [ ]:
with open('mappingTable.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(mapping, file)

In [ ]:
type(mapping)

pandas.core.series.Series

In [ ]:
df2.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'education', 'gPlusUserId', 'address',
       'closed', 'gps', 'hours', 'name', 'phone', 'price', 'categories',
       'gPlusPlaceId', 'rating', 'reviewText', 'reviewCount', 'userCount',
       'jobs', 'loc', 'cleaned_review', 'cleaned_summary', 'cleaned_education',
       'cleaned_address', 'cleaned_hours', 'cleaned_categories',
       'cleaned_jobs', 'sentiment_VADER'],
      dtype='object')

In [ ]:
####### Recommendation without specific category
## Find the most preferred restaurant by person

#def pref_Rest (userID):
#  df1=pd.read_pickle('df1.pkl')
#  bestSent=df1.sort_values(['rating'],ascending=False).groupby(['gPlusUserId'],as_index=False)[['gPlusUserId','name','rating']].first()
#  restName=bestSent["name"].iloc[bestSent.index[bestSent["gPlusUserId"]== userID]].values[0]
#  return restName




In [ ]:
bestSent=df1.sort_values(['rating'],ascending=False).groupby(['gPlusUserId'],as_index=False)[['gPlusUserId','name','rating']].first()
restName=bestSent["name"].iloc[bestSent.index[bestSent["gPlusUserId"]== str(userID)]].values[0]

In [ ]:
df2.to_pickle("df1.pkl")

In [15]:
with open('mappingTable.pkl', 'rb') as file:
      
    # Call load method to deserialze
    mapping = pickle.load(file)
print(mapping)

name
Tamarine                               0
Blue Fig                               1
Umami Burger                           2
BAR CRUDO                              3
Southern Pacific Brewing               4
                                    ... 
Naja's Place                        2084
Crow's Nest Restaurant              2090
The Kitchen Restaurant              2209
Olive Cafe                          2647
Espana's Southwest Bar and Grill    2703
Length: 347, dtype: int64


In [ ]:
category_dummy =df2['categories'].str.join('|').str.get_dummies()
similarity_matrix = linear_kernel(category_dummy,category_dummy)
similarity_matrix

In [ ]:
np.save("similarity_matrix.npy",similarity_matrix)

In [10]:
similarity_matrix = np.load('similarity_matrix.npy')

In [11]:
print(similarity_matrix)

[[3. 1. 1. ... 1. 1. 1.]
 [1. 2. 1. ... 1. 2. 1.]
 [1. 1. 3. ... 1. 1. 2.]
 ...
 [1. 1. 1. ... 3. 1. 1.]
 [1. 2. 1. ... 1. 3. 1.]
 [1. 1. 2. ... 1. 1. 3.]]


In [15]:
def recommend_restaurant(userID=1.002151589e+20):
  df1=pd.read_csv('df_CA_Summarization_filtered.csv')
  with open('mappingTable.pkl', 'rb') as file:
      
    # Call load method to deserialze
    mapping = pickle.load(file)
  similarity_matrix = np.load('similarity_matrix.npy')
  bestSent=df1.sort_values(['rating'],ascending=False).groupby(['gPlusUserId'],as_index=False)[['gPlusUserId','name','sentiment_VADER']].first()
  restaurantChosen=bestSent["name"].iloc[bestSent.index[bestSent["gPlusUserId"]== userID]].values[0]
  restaurant_index = mapping[restaurantChosen]
  #get similarity values with other restaurants
  #similarity_score is the list of index and similarity matrix
  similarity_score = list(enumerate(similarity_matrix[restaurant_index]))
  #sort in descending order the similarity score of movie inputted with all the other movies
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
  # Get the scores of the 15 most similar movies. Ignore the first movie.
  similarity_score = similarity_score[1:4]
  #return movie names using the mapping series
  restaurant_index = [i[0] for i in similarity_score]
  #return ("Top 3 restaurants: 1. "+str(df1['name'].iloc[restaurant_index].values[0])+" 2. "+str(df1['name'].iloc[restaurant_index].values[1])+" 3. "+str(df1['name'].iloc[restaurant_index].values[2]))
  return (pd.DataFrame({'Name':df1['name'].iloc[restaurant_index],'Categories':df1['categories'].iloc[restaurant_index]}))
